In [ ]:
import re
import gurobipy as gb
import numpy as np
import pandas as pd

import networkx as nx
import matplotlib.pyplot as plt
from keplergl import KeplerGl
import geopandas as gpd
from shapely.geometry import Point, LineString

## Loading data


In [ ]:
stops = pd.read_csv("./STM GTFS/stops.txt", sep=",")
stops = gpd.GeoDataFrame(
    stops, geometry=gpd.points_from_xy(stops.stop_lon, stops.stop_lat)
)
stops["stop_code"] = stops["stop_code"].astype(int)

stops

In [ ]:
NUM_ROUTES = 3
distance_matrix = pd.read_json("distance_matrix.json")

In [ ]:
# add depot to distance matrix
for i in range(NUM_ROUTES):
    distance_matrix.loc[i+1] = 0.0
    distance_matrix[i+1] = 0.0

distance_matrix

In [ ]:
np.nanmax(distance_matrix.to_numpy())

In [ ]:
distance_matrix.fillna(1e2, inplace=True)

In [ ]:
distance_matrix

In [ ]:
nodes = list(distance_matrix.index)
num_nodes = len(nodes)

# Vehicle Routing - Bus Route Optimization

In [ ]:
vrp = gb.Model("Montreal Bus Routing")
vrp.Params.MIPGap = 0.3
vrp.Params.TimeLimit = 30
start_node_index = 68


# Decision variables
x = vrp.addVars(nodes, nodes, vtype=gb.GRB.BINARY, name="x")

u = vrp.addVars(nodes, vtype=gb.GRB.INTEGER, name="u", ub=num_nodes)

# Objective function
vrp.setObjective(
    gb.quicksum(distance_matrix.loc[i, j] * x[i, j] for i in nodes for j in nodes),
    gb.GRB.MINIMIZE,
)

# Constraints
vrp.addConstrs(
    (gb.quicksum(x[i, j] for j in nodes if j != i) == 1 for i in nodes),
    name="outgoing",
)

vrp.addConstrs(
    (gb.quicksum(x[j, i] for j in nodes if j != i) == 1 for i in nodes),
    name="ingoing",
)

vrp.addConstrs(
    (
        u[i] - u[j] + num_nodes * x[i, j] <= num_nodes - 1
        for i in nodes
        for j in nodes
        if i != j and i != nodes[start_node_index] and j != nodes[start_node_index]
    ),
    name="subtour_elimination",
)

vrp.addConstrs(
    (u[i] >= 2 for i in nodes if i != nodes[start_node_index]), name="lower bound for u"
)

vrp.addConstr(
    u[nodes[start_node_index]] == 1,
    name="start node index is 1",
)

vrp.optimize()

In [ ]:
nodes[start_node_index]

In [ ]:
print("Minimum distance: ", vrp.objVal)
print("Optimal path: ")
tour = []
for v in vrp.getVars():
    if v.varName.startswith("u"):
        tour.append(v)

for s in sorted(tour, key=lambda x: x.x):
    print(s.varName, s.x)

In [ ]:
# Distances
distances = []

for i in nodes:
    for j in nodes:
        if x[i, j].x == 1:
            distances.append(distance_matrix[i][j])

pd.Series(distances).describe()

In [ ]:
# Plotting the directed graph
G = nx.DiGraph()

for i in nodes:
    for j in nodes:
        if x[i, j].x == 1:
            G.add_edge(
                i, j, weight=distance_matrix[i][j], step=u[i].x, label=f"{i}-{j}"
            )

pos = nx.spring_layout(G)
steps = nx.get_edge_attributes(G, "step")

from pyvis.network import Network

net = Network(
    directed=True,
    notebook=True,
    select_menu=True,
    filter_menu=True,
    cdn_resources="remote",
    neighborhood_highlight=True,
)

net.from_nx(G)
net.show_buttons()


net.show("example.html")

In [ ]:
# convert tour into dictionary of steps and nodes

tour_dict = {}

for node in tour:
    node_name = node.varName

    node_name = re.sub(r"[u\[\]]", "", node_name)
    tour_dict[node.x] = int(node_name)

tour_df = (
    pd.DataFrame.from_dict(tour_dict, orient="index", columns=["node"])
    .sort_index()
    .reset_index()
    .rename(columns={"index": "step"})
)

tour_df = tour_df.merge(stops, left_on="node", right_on="stop_code", how="left")

tour_df["stop_lat"] = tour_df["stop_lat"].fillna(45.5376881)
tour_df["stop_lon"] = tour_df["stop_lon"].fillna(-73.5705049)
tour_df["stop_name"] = tour_df["stop_name"].fillna("Depot")

tour_df["geometry"] = tour_df.apply(
    lambda x: Point((float(x.stop_lon), float(x.stop_lat))), axis=1
)

tour_df = gpd.GeoDataFrame(tour_df, geometry="geometry")

tour_df

In [ ]:
depot_steps = tour_df[tour_df["stop_name"] == "Depot"].index.tolist()

tours = []

for i in range(len(depot_steps) - 1):
    tours.append(tour_df.iloc[depot_steps[i] : depot_steps[i + 1] + 1])

tours.append(tour_df.iloc[depot_steps[-1] :])

In [ ]:
tour_lines = []

for i, tour_ in enumerate(tours):
    tour_lines.append({"route_id": i, "geometry": LineString(tour_.geometry.tolist())})


tour_lines = gpd.GeoDataFrame.from_dict(tour_lines, geometry="geometry")

tour_lines

In [ ]:
config = {
    "version": "v1",
    "config": {
        "visState": {
            "filters": [
                {
                    "dataId": ["routes"],
                    "id": "cc52bgyolv",
                    "name": ["route_id"],
                    "type": "range",
                    "value": [0, 0.73],
                    "enlarged": False,
                    "plotType": "histogram",
                    "animationWindow": "free",
                    "yAxis": None,
                    "speed": 1,
                }
            ],
            "layers": [
                {
                    "id": "w9xelah",
                    "type": "point",
                    "config": {
                        "dataId": "stops_in_trip",
                        "label": "stop",
                        "color": [221, 178, 124],
                        "highlightColor": [252, 242, 26, 255],
                        "columns": {
                            "lat": "stop_lat",
                            "lng": "stop_lon",
                            "altitude": "step",
                        },
                        "isVisible": True,
                        "visConfig": {
                            "radius": 10,
                            "fixedRadius": False,
                            "opacity": 0.8,
                            "outline": False,
                            "thickness": 2,
                            "strokeColor": None,
                            "colorRange": {
                                "name": "Global Warming",
                                "type": "sequential",
                                "category": "Uber",
                                "colors": [
                                    "#5A1846",
                                    "#900C3F",
                                    "#C70039",
                                    "#E3611C",
                                    "#F1920E",
                                    "#FFC300",
                                ],
                            },
                            "strokeColorRange": {
                                "name": "Global Warming",
                                "type": "sequential",
                                "category": "Uber",
                                "colors": [
                                    "#5A1846",
                                    "#900C3F",
                                    "#C70039",
                                    "#E3611C",
                                    "#F1920E",
                                    "#FFC300",
                                ],
                            },
                            "radiusRange": [0, 50],
                            "filled": True,
                        },
                        "hidden": False,
                        "textLabel": [
                            {
                                "field": None,
                                "color": [255, 255, 255],
                                "size": 18,
                                "offset": [0, 0],
                                "anchor": "start",
                                "alignment": "center",
                            }
                        ],
                    },
                    "visualChannels": {
                        "colorField": {"name": "step", "type": "integer"},
                        "colorScale": "quantile",
                        "strokeColorField": None,
                        "strokeColorScale": "quantile",
                        "sizeField": None,
                        "sizeScale": "linear",
                    },
                },
                {
                    "id": "ez5bm5",
                    "type": "geojson",
                    "config": {
                        "dataId": "routes",
                        "label": "routes",
                        "color": [30, 150, 190],
                        "highlightColor": [252, 242, 26, 255],
                        "columns": {"geojson": "geometry"},
                        "isVisible": True,
                        "visConfig": {
                            "opacity": 0.8,
                            "strokeOpacity": 0.8,
                            "thickness": 1,
                            "strokeColor": None,
                            "colorRange": {
                                "name": "Global Warming",
                                "type": "sequential",
                                "category": "Uber",
                                "colors": [
                                    "#5A1846",
                                    "#900C3F",
                                    "#C70039",
                                    "#E3611C",
                                    "#F1920E",
                                    "#FFC300",
                                ],
                            },
                            "strokeColorRange": {
                                "name": "ColorBrewer Paired-5",
                                "type": "qualitative",
                                "category": "ColorBrewer",
                                "colors": [
                                    "#a6cee3",
                                    "#1f78b4",
                                    "#b2df8a",
                                    "#33a02c",
                                    "#fb9a99",
                                ],
                            },
                            "radius": 10,
                            "sizeRange": [0, 10],
                            "radiusRange": [0, 50],
                            "heightRange": [0, 500],
                            "elevationScale": 5,
                            "enableElevationZoomFactor": True,
                            "stroked": True,
                            "filled": False,
                            "enable3d": False,
                            "wireframe": False,
                        },
                        "hidden": False,
                        "textLabel": [
                            {
                                "field": None,
                                "color": [255, 255, 255],
                                "size": 18,
                                "offset": [0, 0],
                                "anchor": "start",
                                "alignment": "center",
                            }
                        ],
                    },
                    "visualChannels": {
                        "colorField": None,
                        "colorScale": "quantile",
                        "strokeColorField": {"name": "route_id", "type": "integer"},
                        "strokeColorScale": "quantile",
                        "sizeField": None,
                        "sizeScale": "linear",
                        "heightField": None,
                        "heightScale": "linear",
                        "radiusField": None,
                        "radiusScale": "linear",
                    },
                },
            ],
            "interactionConfig": {
                "tooltip": {
                    "fieldsToShow": {
                        "stops_in_trip": [
                            {"name": "stop_name", "format": None},
                            {"name": "step", "format": None},
                        ],
                        "routes": [{"name": "route_id", "format": None}],
                    },
                    "compareMode": False,
                    "compareType": "absolute",
                    "enabled": True,
                },
                "brush": {"size": 0.5, "enabled": False},
                "geocoder": {"enabled": False},
                "coordinate": {"enabled": False},
            },
            "layerBlending": "normal",
            "splitMaps": [],
            "animationConfig": {"currentTime": None, "speed": 1},
        },
        "mapState": {
            "bearing": 0,
            "dragRotate": False,
            "latitude": 45.59783403161273,
            "longitude": -73.62435981591366,
            "pitch": 0,
            "zoom": 10,
            "isSplit": False,
        },
        "mapStyle": {
            "styleType": "light",
            "topLayerGroups": {},
            "visibleLayerGroups": {
                "label": True,
                "road": True,
                "border": False,
                "building": True,
                "water": True,
                "land": True,
                "3d building": False,
            },
            "threeDBuildingColor": [
                218.82023004728686,
                223.47597962276103,
                223.47597962276103,
            ],
            "mapStyles": {},
        },
    },
}

In [ ]:
map = KeplerGl(height=700, config=config)

map.add_data(
    data=tour_df[["stop_lat", "stop_lon", "stop_name", "step"]],
    name="stops_in_trip",
)

map.add_data(
    data=tour_lines[['route_id', 'geometry']],
    name="routes",
)

map

## Vehicle Routing Problem - multiple buses